# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.19,168.86,55.49,76,100,4.41,NZ,1596259398
1,Kiryat Gat,31.61,34.76,77.00,55,0,3.00,IL,1596259398
2,Albany,42.60,-73.97,72.00,64,0,3.76,US,1596259344
3,Turmalina,-17.29,-42.73,59.86,91,93,4.27,BR,1596259399
4,Karratha,-20.74,116.85,83.21,17,0,10.04,AU,1596259399
...,...,...,...,...,...,...,...,...,...
572,Mangawān,24.68,81.55,93.09,50,49,6.46,IN,1596259532
573,Elko,41.00,-115.50,80.01,18,1,7.25,US,1596259532
574,San Felipe,10.34,-68.74,72.79,82,70,4.52,VE,1596259532
575,São José da Coroa Grande,-8.90,-35.15,74.21,85,33,10.45,BR,1596259532


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Combine Lat and Lng into new column
cities_df['Lat, Lng'] = list(zip(cities_df['Lat'], cities_df['Lng']))
coordinates = cities_df['Lat, Lng']

# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
weights = cities_df['Humidity']


heat_map = gmaps.heatmap_layer(coordinates, weights=weights)
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# used example code from this url to narrow datafram: 
# https://chrisalbon.com/python/data_wrangling/pandas_selecting_rows_on_conditions/
over_70 = cities_df['Max Temp'] > 72
under_80 = cities_df['Max Temp'] < 76
wind_10 = cities_df['Wind Speed'] < 10
cloud_0 = cities_df['Cloudiness'] == 0

ideal_city_df = cities_df [over_70 & under_80 & wind_10 & cloud_0]
ideal_city_df

lat_lng = ideal_city_df['Lat, Lng'].tolist()
ideal_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,"Lat, Lng"
121,Altay,47.87,88.12,74.64,24,0,6.20,CN,1596259426,"(47.87, 88.12)"
140,Klos,41.51,20.09,75.20,83,0,4.70,AL,1596259430,"(41.51, 20.09)"
195,Upornaya,44.39,41.02,75.87,54,0,4.90,RU,1596259442,"(44.39, 41.02)"
253,Guerrero Negro,27.98,-114.06,72.52,73,0,7.07,MX,1596259437,"(27.98, -114.06)"
286,Yulara,-25.24,130.99,75.20,8,0,5.82,AU,1596259462,"(-25.24, 130.99)"
290,Novo Aripuanã,-5.12,-60.38,72.63,57,0,0.74,BR,1596259463,"(-5.12, -60.38)"
384,Udachny,66.42,112.40,73.40,38,0,4.47,RU,1596259484,"(66.42, 112.4)"
547,Tataouine,32.93,10.45,73.40,60,0,1.12,TN,1596259526,"(32.93, 10.45)"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={g_key}&radius=5000&type=hotel&location='

hotels = []

for location in lat_lng:
    try:
        response = requests.get(url + str(location).replace('(','').replace(')','')).json()
# print(json.dumps(response, indent=4))
        hotels.append(response['results'][0]['name'])
        print('hotel found')
    except:
        print('hotel not found')

hotel found
hotel found
hotel found
hotel found
hotel found
hotel found
hotel found
hotel found


In [18]:
ideal_city_df['Hotel Name'] = hotels

ideal_city_df

C:\Users\jleve\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,"Lat, Lng",Hotel Name
121,Altay,47.87,88.12,74.64,24,0,6.20,CN,1596259426,"(47.87, 88.12)",Altay Prefecture
140,Klos,41.51,20.09,75.20,83,0,4.70,AL,1596259430,"(41.51, 20.09)",Klos
195,Upornaya,44.39,41.02,75.87,54,0,4.90,RU,1596259442,"(44.39, 41.02)",Upornaya
253,Guerrero Negro,27.98,-114.06,72.52,73,0,7.07,MX,1596259437,"(27.98, -114.06)",Guerrero Negro
286,Yulara,-25.24,130.99,75.20,8,0,5.82,AU,1596259462,"(-25.24, 130.99)",Petermann
290,Novo Aripuanã,-5.12,-60.38,72.63,57,0,0.74,BR,1596259463,"(-5.12, -60.38)",Novo Aripuanã
384,Udachny,66.42,112.40,73.40,38,0,4.47,RU,1596259484,"(66.42, 112.4)",Udachny
547,Tataouine,32.93,10.45,73.40,60,0,1.12,TN,1596259526,"(32.93, 10.45)",Tataouine


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_city_df.iterrows()]
locations = ideal_city_df['Lat, Lng']
locations

121      (47.87, 88.12)
140      (41.51, 20.09)
195      (44.39, 41.02)
253    (27.98, -114.06)
286    (-25.24, 130.99)
290     (-5.12, -60.38)
384      (66.42, 112.4)
547      (32.93, 10.45)
Name: Lat, Lng, dtype: object

In [38]:
# Add marker layer ontop of heat map

 # Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…